#Решение задачи классификации набора видеороликов, каждый из которых относится к одному из четырех классов с помощью градиентного бустинга из библиотеки catboost

https://drive.google.com/file/d/1i8RkwwnQ0vzkuNIBLlsLeYi29vsyo5Hs/view?usp=sharing

Загрузим архив с видеороликами с гугл диска

In [ ]:
! gdown --id 1i8RkwwnQ0vzkuNIBLlsLeYi29vsyo5Hs

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1i8RkwwnQ0vzkuNIBLlsLeYi29vsyo5Hs
To: /content/train_sibur_1task.zip
100% 56.0M/56.0M [00:00<00:00, 69.3MB/s]


Распакуем загруженный архив

In [ ]:
! unzip /content/train_sibur_1task.zip

Archive:  /content/train_sibur_1task.zip
   creating: bridge_down/
  inflating: bridge_down/e5f35a69b5fdde5a.mp4  
  inflating: bridge_down/b0b2e92153ee21f3.mp4  
  inflating: bridge_down/0a9d37c222ccce4a.mp4  
  inflating: bridge_down/0cbedb20b827e285.mp4  
  inflating: bridge_down/354914dca502d24e.mp4  
  inflating: bridge_down/9b38da83fc283459.mp4  
  inflating: bridge_down/c98b9d9ce7d20c67.mp4  
  inflating: bridge_down/65e1f3ca7f161cf3.mp4  
  inflating: bridge_down/47f02ea5b361b905.mp4  
  inflating: bridge_down/78988e7b0fa6d3b6.mp4  
  inflating: bridge_down/84440a85bdcda906.mp4  
  inflating: bridge_down/100816388bee023c.mp4  
  inflating: bridge_down/5f1fc70dd930375f.mp4  
  inflating: bridge_down/2a5e4a3cddf72998.mp4  
  inflating: bridge_down/ab9dfd1072d4e259.mp4  
  inflating: bridge_down/790296a321decd6b.mp4  
  inflating: bridge_down/3884f8809fe08a41.mp4  
  inflating: bridge_down/efa044318bc83065.mp4  
  inflating: bridge_down/6edfb18ca69cae9c.mp4  
  inflating: bridge_d

Загрузим необходимые библиотеки.

In [ ]:
import cv2
import pandas as pd
import os
import numpy as np
import pathlib
import warnings
warnings.filterwarnings("ignore")


In [ ]:
RS = 3984765

\обработаем каждую папку с файлами mp4 и и переведем его в датасет с картинками

In [ ]:
labels = ["bridge_down", "bridge_up", "no_action", "train_in_out"]

In [ ]:
train_clips = {"label":[], "fname": []}

for label in labels:
    path = '/content/'
    list_files = os.listdir(path+label)
    for file_name in list_files:
        train_clips['label'].append(label)
        train_clips['fname'].append(file_name)

train_clips = pd.DataFrame(train_clips)
train_clips

,label,fname
0,bridge_down,98861bd2978d75b6.mp4
1,bridge_down,fa47b56df3bdfa7c.mp4
2,bridge_down,73297e384cf9b2b0.mp4
3,bridge_down,cca3a1c36a9e7f36.mp4
4,bridge_down,790296a321decd6b.mp4
...,...,...
491,train_in_out,5564ec76ba7dcc48.mp4
492,train_in_out,15a80e7c7a533d3d.mp4
493,train_in_out,088d91a5d5de25f4.mp4
494,train_in_out,167fbd4c45163b18.mp4


Посмотрим сколько роликов каждого класса находится в датасете

In [ ]:
train_clips.label.value_counts()

bridge_down     306
bridge_up        75
train_in_out     66
no_action        49
Name: label, dtype: int64

Набор данных не сбалансирован

Напишем функцию, которая будет делать расскадровку видеороликов

In [ ]:
def read_clip(odir: pathlib.Path, fname: str, start: int = 0, transposed: bool = True):
    """Прочесть ролик в массив."""

    cpr = cv2.VideoCapture(odir.joinpath(fname).as_posix())
    has_frame = True
    frames = []

    while has_frame:
        has_frame, frame = cpr.read()
        if has_frame:
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            if transposed:
                frame = np.moveaxis(frame, -1, 0).copy()

            frames.append(frame)
    cpr.release()
    return np.array(frames)[start:]

С помощью претренированной модели извлечём признаки для каждого кадра,
посчитаем среднее значение для всего ролика,
натренируем простую модель на полученных признаках.
Для извлечения признаков используем какую-либо из моделей, доступных в TensorFlow Hub. 
Для ускорения будем уменьшать изображения до 96 x 96:

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
fts_extract = tf.keras.Sequential([
    tf.keras.layers.Resizing(96, 96, interpolation="bilinear"),
    tf.keras.layers.Rescaling(scale=1.0 / 127.5, offset=-1),
    hub.KerasLayer("https://tfhub.dev/google/imagenet/mobilenet_v2_100_96/feature_vector/5", trainable=False)
])
fts_extract.build([None, 240, 320, 3])

У нас всего 496 роликов, поэтому 1280 признаков нам не нужны (и избыточны). Однако и нормально провести dimensionality reduction при таком количестве признаков тоже непросто. Поэтому сделаем random projection:

In [ ]:
from sklearn.random_projection import GaussianRandomProjection

In [ ]:
from sklearn.model_selection import train_test_split
projector = GaussianRandomProjection(n_components=50, random_state=RS)
projector.fit(np.random.rand(10, 1280))

GaussianRandomProjection(n_components=50, random_state=3984765)

In [ ]:
import tqdm
DATA_DIR = pathlib.Path("")

Вытащим признаки из набора видеороликов, предварительно сделав раскадровку с помощью функции read_clip, представленной выше:

In [ ]:
features = []
for ci, row in tqdm.tqdm(train_clips.iterrows(), total=train_clips.shape[0]):
    clip = read_clip(DATA_DIR.joinpath("/content/", row.label), row.fname, transposed=False)
    outputs = fts_extract(clip)
    features.append(projector.transform(outputs.numpy()).mean(axis=0))

100%|██████████| 496/496 [17:48<00:00,  2.15s/it]


In [ ]:
features = np.vstack(features)

Соберем признаки в DataFrame

In [ ]:
df = pd.DataFrame(features, index=train_clips.index, columns=[f"fts{i}" for i in range(features.shape[1])])
df

,fts0,fts1,fts2,fts3,fts4,fts5,fts6,fts7,fts8,fts9,...,fts40,fts41,fts42,fts43,fts44,fts45,fts46,fts47,fts48,fts49
0,-2.549675,-0.761511,0.439506,-2.572969,-7.088608,-5.110124,-1.561289,-4.117859,5.731224,-3.939584,...,4.973745,5.138914,1.962427,3.440993,2.735537,7.284228,0.259979,1.769667,10.133673,0.771189
1,-2.267458,1.841458,8.713071,1.015860,-6.367874,2.607790,-1.218254,-2.096806,2.053915,4.249879,...,4.374151,6.049245,-1.376278,-0.558588,0.103114,6.474805,4.923724,-0.269710,7.037655,2.638213
2,1.359956,-3.710513,8.610892,-0.828716,-2.725628,-5.660552,-6.859534,1.537932,8.448419,-2.828681,...,2.131435,6.845980,-0.725789,-4.082477,5.229553,3.871656,12.051599,-1.527519,8.983080,0.723137
3,-2.489228,-4.238406,4.764355,-5.242531,-4.692200,-4.000678,-5.828839,1.254182,3.792231,-0.010511,...,-0.696800,3.659305,-2.648019,-0.819664,6.144162,1.545190,9.141098,-2.119478,6.135501,3.167746
4,5.789810,-4.875088,5.849388,-3.530506,-9.633269,1.122845,-6.887261,4.212868,-0.623483,-1.036255,...,0.218073,-1.265042,-4.009845,-0.778813,2.452328,4.310110,7.055913,-1.846539,1.614427,-5.178898
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
491,7.755018,-5.064724,3.008095,-5.581529,-5.439809,0.805191,-3.075935,-1.228055,6.822593,-0.247532,...,5.699385,0.838541,-3.403297,-1.557353,2.812439,4.900637,9.102808,-2.594538,2.831916,1.811559
492,-2.637232,-4.088146,5.116719,-1.845585,-2.646413,1.156945,0.875141,2.804753,6.853201,-1.861731,...,-6.604254,6.688292,-0.755443,-1.820704,6.673156,2.860103,-0.651445,-5.604997,5.850603,-1.029724
493,9.476911,-5.390709,4.950569,-3.290462,-8.733536,2.628379,-0.649334,1.401924,4.197980,-1.301183,...,8.366649,1.493234,-3.585271,2.443919,2.912973,5.418749,12.082981,-3.480458,5.610549,0.279700
494,10.443397,-8.519401,5.307245,0.881264,-0.804222,3.138812,-2.891654,1.427912,2.523178,0.418698,...,0.076511,-3.150720,-3.864765,-2.425451,6.007544,4.089101,3.805128,-6.297423,-1.903657,2.106141


Соединим по индексам полученный датафрейм с train_clips

In [ ]:

joined_df = pd.concat([train_clips, df], axis=1)
df = joined_df.iloc[:, :]
df.head()

,label,fname,fts0,fts1,fts2,fts3,fts4,fts5,fts6,fts7,...,fts40,fts41,fts42,fts43,fts44,fts45,fts46,fts47,fts48,fts49
0,bridge_down,98861bd2978d75b6.mp4,-2.549675,-0.761511,0.439506,-2.572969,-7.088608,-5.110124,-1.561289,-4.117859,...,4.973745,5.138914,1.962427,3.440993,2.735537,7.284228,0.259979,1.769667,10.133673,0.771189
1,bridge_down,fa47b56df3bdfa7c.mp4,-2.267458,1.841458,8.713071,1.015860,-6.367874,2.607790,-1.218254,-2.096806,...,4.374151,6.049245,-1.376278,-0.558588,0.103114,6.474805,4.923724,-0.269710,7.037655,2.638213
2,bridge_down,73297e384cf9b2b0.mp4,1.359956,-3.710513,8.610892,-0.828716,-2.725628,-5.660552,-6.859534,1.537932,...,2.131435,6.845980,-0.725789,-4.082477,5.229553,3.871656,12.051599,-1.527519,8.983080,0.723137
3,bridge_down,cca3a1c36a9e7f36.mp4,-2.489228,-4.238406,4.764355,-5.242531,-4.692200,-4.000678,-5.828839,1.254182,...,-0.696800,3.659305,-2.648019,-0.819664,6.144162,1.545190,9.141098,-2.119478,6.135501,3.167746
4,bridge_down,790296a321decd6b.mp4,5.789810,-4.875088,5.849388,-3.530506,-9.633269,1.122845,-6.887261,4.212868,...,0.218073,-1.265042,-4.009845,-0.778813,2.452328,4.310110,7.055913,-1.846539,1.614427,-5.178898


Разделим данные на X и y

In [ ]:

X = df.filter(like="fts")
y = df.label


Разделим выборку на тренировочную и валидационную

In [ ]:
X_tr, X_val, y_tr, y_val = train_test_split(X, y,
                                                    test_size=0.2, random_state=RS)

Установим библиотеку catboost

In [ ]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 MB 2.4 MB/s eta 0:00:00


Будем использовать градиентный бустинг, поскольку во многих случаях он позволяет получить хорошие показатели даже без оптимизации модели. Попробуем натренировать CatBoostClassifier на этих признаках.

In [ ]:
from catboost import CatBoostClassifier

In [ ]:
from sklearn.metrics import f1_score, classification_report

In [ ]:
cb_model = CatBoostClassifier(max_depth=4, iterations=1000, objective = 'MultiClass', verbose=200, eval_metric = 'TotalF1',
                                 random_state=RS)
cb_model.fit(X_tr, y_tr, eval_set=(X_val, y_val))
val_preds = cb_model.predict(X_val)
print(classification_report(y_val, val_preds))

Learning rate set to 0.105994
0:	learn: 0.6287545	test: 0.5731905	best: 0.5731905 (0)	total: 14.2ms	remaining: 14.1s
200:	learn: 0.9949138	test: 0.8108190	best: 0.8108190 (177)	total: 3.69s	remaining: 14.7s
400:	learn: 1.0000000	test: 0.8330718	best: 0.8330718 (224)	total: 5.92s	remaining: 8.85s
600:	learn: 1.0000000	test: 0.8338289	best: 0.8338289 (598)	total: 8.15s	remaining: 5.41s
800:	learn: 1.0000000	test: 0.8330718	best: 0.8439810 (662)	total: 10.4s	remaining: 2.58s
999:	learn: 1.0000000	test: 0.8439810	best: 0.8439810 (662)	total: 12.5s	remaining: 0us

bestTest = 0.8439809524
bestIteration = 662

Shrink model to first 663 iterations.
              precision    recall  f1-score   support

 bridge_down       0.85      0.97      0.90        59
   bridge_up       0.88      0.74      0.80        19
   no_action       1.00      0.75      0.86         8
train_in_out       0.73      0.57      0.64        14

    accuracy                           0.85       100
   macro avg       0.86  

Сохраним модель в файл

In [ ]:
from joblib import dump, load
dump(cb_model, 'classifier-v1.joblib')
dump(projector, 'projector-v1.joblib')

['projector-v1.joblib']

Подготовим сабмит и запишем файл предикт

In [ ]:
%%writefile predict.py

import pathlib
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
from joblib import load

PROJECTOR_FILE = pathlib.Path(__file__).parent.joinpath("projector-v1.joblib")
CLASSIFIER_FILE = pathlib.Path(__file__).parent.joinpath("classifier-v1.joblib")


def construct_model():
    fts_extract = tf.keras.Sequential([
        tf.keras.layers.Resizing(96, 96, interpolation="bilinear"),
        tf.keras.layers.Rescaling(scale=1.0 / 127.5, offset=-1),
        hub.KerasLayer("https://tfhub.dev/google/imagenet/mobilenet_v2_100_96/feature_vector/5", trainable=False)
    ])
    fts_extract.build([None, 240, 320, 3])
    return fts_extract


model = construct_model()
projector = load(PROJECTOR_FILE)
classifier = load(CLASSIFIER_FILE)


def predict(clip: np.ndarray):
    """Вычислить класс для этого клипа. Эта функция должна возвращать *имя* класса."""

    features = projector.transform(model(clip).numpy()).mean(axis=0, keepdims=True)
    return classifier.predict(features)[0]

Writing predict.py


Файл launch.json позволяет загрузить веса при сборке контейнера:

In [ ]:
%%writefile launch.json
{
    "torch": ["shufflenet_v2_x0_5"],
    "tfhub": ["https://tfhub.dev/google/imagenet/mobilenet_v2_100_96/feature_vector/5"],
    "transposed": false
}

Writing launch.json


Запишем файл requirements.txt с версиями используемых библиотек.

In [ ]:
!pip3 freeze > requirements.txt